<a href="https://colab.research.google.com/github/cgelil/cgelil.github.io/blob/main/CC6_loops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#pip install dtmapi

In [6]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import altair as alt
import geopandas as gpd
import matplotlib.pyplot as plt
import json
import altair as alt
import dtmapi
import statsmodels.api as sm
from statsmodels.formula.api import ols
import os

In [8]:
all_country_list = ['Afghanistan', 'Democratic Republic of the Congo', 'Ethiopia', 'Iraq', 'Nigeria', 'Somalia', 'South Sudan', 'Sudan', 'Yemen']

# Initialize an empty list to store dataframes
all_dataframes = []

# Loop through each country in the list
for country in all_country_list:
    # Fetch data for each country
    df = dtmapi.get_idp_admin0_data(CountryName=country, FromReportingDate='2010-01-01', ToReportingDate='2024-12-31', to_pandas=True)
    all_dataframes.append(df)

dtm_df = pd.concat(all_dataframes, ignore_index=True)
dtm_df

,id,operation,admin0Name,admin0Pcode,numPresentIdpInd,reportingDate,yearReportingDate,monthReportingDate,roundNumber,assessmentType
0,8641,Returns and displacement in Afghanistan,Afghanistan,AFG,246362,2017-03-30T00:00:00,2017,3,1,BA
1,8642,Returns and displacement in Afghanistan,Afghanistan,AFG,945182,2017-06-30T00:00:00,2017,6,2,BA
2,8643,Returns and displacement in Afghanistan,Afghanistan,AFG,1728157,2017-12-31T00:00:00,2017,12,3,BA
3,8644,Returns and displacement in Afghanistan,Afghanistan,AFG,1843833,2018-03-30T00:00:00,2018,3,4,BA
4,8645,Returns and displacement in Afghanistan,Afghanistan,AFG,1866925,2018-06-30T00:00:00,2018,6,5,BA
...,...,...,...,...,...,...,...,...,...,...
601,8868,Yemen conflict,Yemen,YEM,612132,2018-05-30T00:00:00,2018,5,31,BA
602,8869,Yemen conflict,Yemen,YEM,2331264,2018-06-30T00:00:00,2018,6,32,BA
603,8870,Yemen conflict,Yemen,YEM,3647250,2018-11-30T00:00:00,2018,11,37,BA
604,8871,Yemen conflict,Yemen,YEM,2302346,2022-11-30T00:00:00,2022,11,38,BA


In [9]:
# Convert reportingDate to datetime
dtm_df['reportingDate'] = pd.to_datetime(dtm_df['reportingDate'])
dtm_df['year'] = dtm_df['reportingDate'].dt.year
idx = dtm_df.groupby(['admin0Name', 'year'])[['monthReportingDate', 'roundNumber']].idxmax()
most_recent_data_per_year = dtm_df.loc[idx['monthReportingDate']]
most_recent_data_per_year.reset_index(drop=True, inplace=True)


# filter the top 10 countries with the highest number of IDPs
countries = most_recent_data_per_year.groupby('admin0Name')['numPresentIdpInd'].sum().nlargest(10).index
countries_data = most_recent_data_per_year[most_recent_data_per_year['admin0Name'].isin(countries)]
countries_data = countries_data[countries_data['year'] >= 2014]

countries_data.to_json('countries_data.json', orient='records')